In [3]:
import sys
sys.path.insert(0,'/p/project/training2005/jupyter/kernels/tensorflow_test/lib/python3.6/site-packages')
import tensorflow

import tensorflow.keras as keras
from keras import layers, Input, Model
from functools import reduce

import numpy as np
ttd = np.load('/p/project/training2005/HZG_Challenge/tas_train.npy')
ptd = np.load('/p/project/training2005/HZG_Challenge/psl_train.npy')
ntd = np.load('/p/project/training2005/HZG_Challenge/nao_index_train.npy')

ttst = np.load('/p/project/training2005/HZG_Challenge/tas_predict.npy')
ptst = np.load('/p/project/training2005/HZG_Challenge/psl_predict.npy')

#plt.plot(range(data.shape[1]),data[0])

def _add_layer(dim, input_tensor, activ):
    if activ != '':
        x = layers.Dense(dim, activation=activ)(input_tensor)
    else: 
        x = layers.Dense(dim)(input_tensor)
    x = layers.BatchNormalization()(x)
    return x

def dense_cell(dim, input_tensor, activs):
    x = _add_layer(dim, input_tensor, activs[0])
    for a in activs[1:]:
        x = _add_layer(dim,x,a)
        
    return x


def cnn_cell(x, ks, st, filters):
    x_shortcut = x
    x = layers.Conv1D(filters=filters[0], kernel_size=1, strides=st, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(filters=filters[1], kernel_size=ks, strides=1, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(filters=filters[2], kernel_size=1, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(filters=1, kernel_size=1, strides=1, padding='same')(x)
    x = layers.UpSampling1D(size=2322)(x)
    x = layers.Add()([x,x_shortcut])
    x = layers.BatchNormalization()(x)
    return x

def build_model():
    inp1 = Input(shape=(2322,1))
    inp2 = Input(shape=(2322,1))
    #out = layers.Dense(1)(inp)
    
    #first branch
    cx = cnn_cell(inp1, 2, 1, [128,128,128])
    x = dense_cell(512, cx, ['relu']*5 + [''])
    
    #second branch
    cy = cnn_cell(inp2, 2, 1, [128,128,128])
    y = dense_cell(512, cy, ['relu']*5 + [''])
    
    #merge branches
    out = layers.Add()([x,y])
    out = layers.Flatten()(out)
    out = layers.Dense(1)(out)
    model = Model([inp1,inp2], out)
  
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mae'])
    model.summary()
    return model

def train_model(model, params ={'vsplit':0.111,'ne':15,'bs':128}):
#     d1 = 
    history = model.fit([np.reshape(ttd,(ttd.shape[0],ttd.shape[1],1)),
                         np.reshape(ptd,(ptd.shape[0],ptd.shape[1],1))], ntd,
                        epochs=params['ne'], 
                        batch_size=params['bs'],
                        validation_split=params['vsplit'])
    return [model, history]

def test_model(model):
    return model.predict([ttst,ptst])

Using TensorFlow backend.


In [2]:
model = build_model()

AttributeError: 'RepeatedCompositeFieldContainer' object has no attribute 'append'

In [ ]:
model, train_hist = train_model(model, params={'vsplit':0.111,'ne':15,'bs':128})

In [ ]:
predictions = test_model(model)